In [25]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [26]:
!!curl -O http://www.manythings.org/anki/rus-eng.zip
!!unzip rus-eng.zip

['Archive:  rus-eng.zip',
 'replace rus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n']

In [27]:
batch_size = 64
epochs = 20
latent_dim = 256
num_samples = 10000
data_path = "rus.txt"

In [28]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [29]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save("s2s")

Epoch 1/20
125/125 [==============================] - 53s 400ms/step - loss: 0.9948 - accuracy: 0.7784 - val_loss: 0.9028 - val_accuracy: 0.7644
Epoch 2/20
125/125 [==============================] - 49s 393ms/step - loss: 0.7158 - accuracy: 0.8127 - val_loss: 0.6954 - val_accuracy: 0.8154
Epoch 3/20
125/125 [==============================] - 49s 392ms/step - loss: 0.6021 - accuracy: 0.8358 - val_loss: 0.5966 - val_accuracy: 0.8313
Epoch 4/20
125/125 [==============================] - 49s 390ms/step - loss: 0.5216 - accuracy: 0.8523 - val_loss: 0.5499 - val_accuracy: 0.8383
Epoch 5/20
125/125 [==============================] - 49s 395ms/step - loss: 0.4826 - accuracy: 0.8610 - val_loss: 0.5127 - val_accuracy: 0.8522
Epoch 6/20
125/125 [==============================] - 48s 381ms/step - loss: 0.4536 - accuracy: 0.8679 - val_loss: 0.4991 - val_accuracy: 0.8533
Epoch 7/20
125/125 [==============================] - 48s 382ms/step - loss: 0.4312 - accuracy: 0.8742 - val_loss: 0.4766 - val_ac

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [31]:
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    return decoded_sentence

In [33]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Подожди!

-
Input sentence: Go.
Decoded sentence: Подожди!

-
Input sentence: Go.
Decoded sentence: Подожди!

-
Input sentence: Hi.
Decoded sentence: Улибой.

-
Input sentence: Hi.
Decoded sentence: Улибой.

-
Input sentence: Hi.
Decoded sentence: Улибой.

-
Input sentence: Hi.
Decoded sentence: Улибой.

-
Input sentence: Hi.
Decoded sentence: Улибой.

-
Input sentence: Hi.
Decoded sentence: Улибой.

-
Input sentence: Run!
Decoded sentence: Беги!

-
Input sentence: Run!
Decoded sentence: Беги!

-
Input sentence: Run.
Decoded sentence: Не чина.

-
Input sentence: Run.
Decoded sentence: Не чина.

-
Input sentence: Who?
Decoded sentence: Кто такой виня?

-
Input sentence: Wow!
Decoded sentence: Просовай!

-
Input sentence: Wow!
Decoded sentence: Просовай!

-
Input sentence: Wow!
Decoded sentence: Просовай!

-
Input sentence: Wow!
Decoded sentence: Просовай!

-
Input sentence: Wow!
Decoded sentence: Просовай!

-
Input sentence: Wow!
Decoded sentence: